In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras import layers, ops
from sklearn.model_selection import train_test_split

2025-12-19 15:10:01.289749: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-19 15:10:01.290996: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-19 15:10:01.509639: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-19 15:10:06.888402: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
## Config a fournir aux etudiants.
maxlen = 128
embed_dim = 128
num_heads = 4
ff_dim = 512

1. Chargez votre dataset, et previsualisez un echantillon de celui-ci.

In [3]:
csv_path = "dataset.csv"

df = pd.read_csv(csv_path)

df = df.dropna(subset=["seq", "sst3"]).reset_index(drop=True)
df = df[df["seq"].str.len() == df["sst3"].str.len()].reset_index(drop=True)

sequences = df["seq"].astype(str).tolist()
labels_ss = df["sst3"].astype(str).tolist()

print(df.head(5))


  pdb_id chain_code                              seq  \
0   1MZW          B  EVKASLRALGEPITLFGEGPAERRERLRNIL   
1   1P9I          A  MDQLNALLASLEAENKQLKAKVEELLAKVGE   
2   3HFE          B  GARGSNTIGARLNRVEDKVTQLDQRLALITD   
3   3LJM          C  *EWEALEKKCAALESKLQALEKKLEALEHG*   
4   4PND          B  *GNILQKIENILKKIENILWKIENILQKIEG   

                              sst3  len  
0  CHHHHHHHCCCCCEECCCCHHHHHHHHHHHC   31  
1  CCCHHHHHHHHHHHHHHHHHHHHHHHHHHCC   31  
2  CCCCCCCHHHHHHHHHHHHHHHHHHHHHHCC   31  
3  CCHHHHHHHHHHHHHHHHHHHHHHHHHHCCC   31  
4  CCCHHHHHHHHHHHHHHHHHHHHHHHHHHHC   31  


2. Encodez les acides amines ainsi que les structures secondaires correspondantes.

In [4]:
aa_chars = sorted({c for seq in sequences for c in seq})

aa_to_idx = {c: i + 1 for i, c in enumerate(aa_chars)}
idx_to_aa = {i + 1: c for i, c in enumerate(aa_chars)}
vocab_size = len(aa_to_idx) + 1

ss_to_idx = {"H": 0, "E": 1, "C": 2}
idx_to_ss = {v: k for k, v in ss_to_idx.items()}
num_classes = 3

encoded_seqs = [[aa_to_idx.get(c, 0) for c in seq] for seq in sequences]

encoded_ss = [[ss_to_idx.get(c, 2) for c in ss] for ss in labels_ss]

3. Preparez vos vecteurs de train et prediction (X et y).

In [5]:
X = keras.utils.pad_sequences(
    encoded_seqs,
    maxlen=maxlen,
    padding="post",
    truncating="post",
    value=0,
)

y = keras.utils.pad_sequences(
    encoded_ss,
    maxlen=maxlen,
    padding="post",
    truncating="post",
    value=-1,
)

x_train, x_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

4. Creez une fonction transformer_block

In [6]:
def transformer_block(x, embed_dim, num_heads, ff_dim, rate=0.1, name="tb"):
    attn = layers.MultiHeadAttention(
        num_heads=num_heads,
        key_dim=embed_dim,
        name=f"{name}_mha",
    )(x, x)

    attn = layers.Dropout(rate, name=f"{name}_drop1")(attn)
    out1 = layers.LayerNormalization(epsilon=1e-6, name=f"{name}_ln1")(x + attn)

    ffn = layers.Dense(ff_dim, activation="relu", name=f"{name}_ff1")(out1)
    ffn = layers.Dense(embed_dim, name=f"{name}_ff2")(ffn)
    ffn = layers.Dropout(rate, name=f"{name}_drop2")(ffn)

    out2 = layers.LayerNormalization(epsilon=1e-6, name=f"{name}_ln2")(out1 + ffn)
    return out2

6. Creez une fonction token_and_position_embedding

In [7]:
def token_and_position_embedding(x, maxlen, vocab_size, embed_dim, name="emb"):
    tok = layers.Embedding(
        input_dim=vocab_size,
        output_dim=embed_dim,
        mask_zero=True,
        name=f"{name}_tok",
    )(x)

    seq_len = ops.shape(x)[-1]
    positions = ops.arange(0, seq_len, 1)
    pos = layers.Embedding(
        input_dim=maxlen,
        output_dim=embed_dim,
        name=f"{name}_pos",
    )(positions)

    return tok + pos

7. Implementez votre modele avec un layer d'embedding puis deux block Transformers consecutifs et un layer dense de sortie s'appuyant sur la fonction d'activation softmax. Vous pourrez afficher votre model avec model.summary() 

In [8]:
inputs = keras.Input(shape=(maxlen,))

x = token_and_position_embedding(inputs, maxlen, vocab_size, embed_dim, name="emb")

x = transformer_block(x, embed_dim, num_heads, ff_dim, rate=0.1, name="tb1")
x = transformer_block(x, embed_dim, num_heads, ff_dim, rate=0.1, name="tb2")

outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="transformer")
model.summary()

E0000 00:00:1766153410.397148 1193247 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1766153410.493830 1193247 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ emb_tok (Embedding) │ (None, 128, 128)  │      2,816 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 128, 128)  │          0 │ emb_tok[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb1_mha             │ (None, 128, 128)  │    263,808 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb1_drop1 (Dropout) │ (None, 128, 128)  │          0 │ tb1_mha[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 128, 128)  │          0 │ add[0][0],        │
│                     │                   │            │ tb1_drop1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb1_ln1             │ (None, 128, 128)  │        256 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb1_ff1 (Dense)     │ (None, 128, 512)  │     66,048 │ tb1_ln1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb1_ff2 (Dense)     │ (None, 128, 128)  │     65,664 │ tb1_ff1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb1_drop2 (Dropout) │ (None, 128, 128)  │          0 │ tb1_ff2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 128, 128)  │          0 │ tb1_ln1[0][0],    │
│                     │                   │            │ tb1_drop2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb1_ln2             │ (None, 128, 128)  │        256 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb2_mha             │ (None, 128, 128)  │    263,808 │ tb1_ln2[0][0],    │
│ (MultiHeadAttentio… │                   │            │ tb1_ln2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb2_drop1 (Dropout) │ (None, 128, 128)  │          0 │ tb2_mha[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 128, 128)  │          0 │ tb1_ln2[0][0],    │
│                     │                   │            │ tb2_drop1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb2_ln1             │ (None, 128, 128)  │        256 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb2_ff1 (Dense)     │ (None, 128, 512)  │     66,048 │ tb2_ln1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb2_ff2 (Dense)     │ (None, 128, 128)  │     65,664 │ tb2_ff1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tb2_drop2 (Dropout) │ (None, 128, 128)  │          0 │ tb2_ff2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 128, 128)  │          0 │ tb2_ln1[0][0],    │
│                     │                   │            │ tb2_drop2[0][0] 

 Total params: 795,267 (3.03 MB)

 Trainable params: 795,267 (3.03 MB)

 Non-trainable params: 0 (0.00 B)

8. Redigez deux methodes loss et accuracy afin d'ignorer le padding des sequences passees dans le modele.

In [9]:
def loss(y_true, y_pred):
    mask = ops.cast(y_true >= 0, "float32")
    y_true_clipped = ops.maximum(y_true, 0)

    scce = keras.losses.SparseCategoricalCrossentropy(
        from_logits=False,
        reduction="none",
    )
    loss_per_token = scce(y_true_clipped, y_pred)
    loss_per_token = loss_per_token * mask

    return ops.sum(loss_per_token) / (ops.sum(mask) + 1e-9)


def accuracy(y_true, y_pred):
    mask = ops.cast(y_true >= 0, "float32")
    y_true_clipped = ops.maximum(y_true, 0)

    y_pred_labels = ops.argmax(y_pred, axis=-1)
    matches = ops.cast(ops.equal(y_true_clipped, y_pred_labels), "float32")
    matches = matches * mask
    return ops.sum(matches) / (ops.sum(mask) + 1e-9)

9. Preparez des callbacks (earlyStopping, ajustement du learning rate), puis compilez votre modele et entrainez le.

In [10]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        min_lr=1e-6,
    )
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=loss,
    metrics=[accuracy],
)

history = model.fit(
    x_train,
    y_train,
    batch_size=42,
    epochs=30,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 67s 454ms/step - accuracy: 0.4991 - loss: 1.0245 - val_accuracy: 0.5030 - val_loss: 1.0011 - learning_rate: 3.0000e-04
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 47s 457ms/step - accuracy: 0.5308 - loss: 0.9610 - val_accuracy: 0.5374 - val_loss: 0.9540 - learning_rate: 3.0000e-04
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 45s 438ms/step - accuracy: 0.5366 - loss: 0.9518 - val_accuracy: 0.5390 - val_loss: 0.9473 - learning_rate: 3.0000e-04
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 46s 442ms/step - accuracy: 0.5397 - loss: 0.9460 - val_accuracy: 0.5415 - val_loss: 0.9457 - learning_rate: 3.0000e-04
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 45s 434ms/step - accuracy: 0.5448 - loss: 0.9390 - val_accuracy: 0.5308 - val_loss: 0.9596 - learning_rate: 3.0000e-04
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 43s 420ms/step - accuracy: 0.5432 - loss: 0.9393 - val_accuracy: 0.5430 - val_loss: 0.9417 - learning_rate: 3.0000e-04
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 43s 41

10. Enregistrez les courbes d'apprentissage dans training_panels.png

In [11]:
def plot_history_panels(history, out_path="training_panels.png"):

    train_loss = history.history.get("loss", [])
    val_loss = history.history.get("val_loss", [])
    train_acc = history.history.get("accuracy", [])
    val_acc = history.history.get("val_accuracy", [])

    epochs = range(1, len(train_loss) + 1)

    fig, axs = plt.subplots(2, 1, figsize=(8, 8))

    ax = axs[0]
    ax.plot(epochs, train_loss, label="Train loss")
    ax.plot(epochs, val_loss, label="Val loss", linestyle="--")
    ax.set_title("Loss")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Loss")
    ax.grid(True)
    ax.legend()

    ax = axs[1]
    ax.plot(epochs, train_acc, label="Train accuracy")
    ax.plot(epochs, val_acc, label="Val accuracy", linestyle="--")
    ax.set_title("Accuracy")
    ax.set_xlabel("Epochs")
    ax.set_ylabel("Accuracy")
    ax.grid(True)
    ax.legend()

    plt.tight_layout()
    plt.savefig(out_path, dpi=150)
    plt.close()
    print(f"Panel sauvegardé sous : {out_path}")

plot_history_panels(history, out_path="training_panels.png")

Panel sauvegardé sous : training_panels.png


11. Ecrire une fonction permettant de tirer un echantillon au hasard de notre dataset, et d'effectuer une prediction a partir de la sequence d'AA. Comparez votre prediction avec la ss3 reelle.

In [12]:
def predict_random_sequence(model, df, maxlen, aa_to_idx, idx_to_ss):
    row = df.sample(1).iloc[0]
    seq = row["seq"]
    true_ss = row["sst3"]

    print("\n=== Exemple tiré au hasard ===")
    print("Séquence :", seq)
    print("Structure réelle :", true_ss)

    encoded = [aa_to_idx.get(c, 0) for c in seq]
    encoded = keras.utils.pad_sequences(
        [encoded], maxlen=maxlen, padding="post", truncating="post"
    )

    pred = model.predict(encoded)[0]
    pred_labels = pred.argmax(axis=-1)
    pred_ss3 = "".join(idx_to_ss[i] for i in pred_labels)

    print("Structure prédite :", pred_ss3[:len(seq)])
    print("=============================\n")

    return seq, true_ss, pred_ss3[:len(seq)]


predict_random_sequence(model, df, maxlen, aa_to_idx, idx_to_ss)


=== Exemple tiré au hasard ===
Séquence : GDCTGKLRGNVAANKETTFQGLTIASGARESEKVFAQTVLSHV
Structure réelle : CCCCCCCCCCCCCCCCEEECCEEEEECCCHHHHHHHHHHHHHC
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
Structure prédite : CCCCCHHHCHHHHHHHHHHHCHHHHCCHHHHHHHHHHHHHHHH



('GDCTGKLRGNVAANKETTFQGLTIASGARESEKVFAQTVLSHV',
 'CCCCCCCCCCCCCCCCEEECCEEEEECCCHHHHHHHHHHHHHC',
 'CCCCCHHHCHHHHHHHHHHHCHHHHCCHHHHHHHHHHHHHHHH')